In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
from itertools import product
import time as t

In [7]:
from da_utils import plot_diff_all, plot_gem0_scan, write_gem0_offline

In [10]:
xlabels = ['te_value', 'ti_value', 'te_ddrho', 'ti_ddrho']
ylabels = ['te_transp_flux', 'ti_transp_flux']

n_fts=8

In [11]:
### To create GEM0 scan files:

date_now = '20240129'

#file_in = "../../muscle3/ref_train_data_5000.csv"
#file_in = "gem0_new_data_20231215.csv"
#file_in = "gem0py_lhc_20240110_all.csv"

# coming back to 5000 full grid GEM0 data
#file_in = "gem0_new_data_20231208.csv"
# ... but on corrected radial locations
#file_in = "gem0_new_data_20231215.csv"
# ... newly generated such reading
file_in = 'gem0py_new_20240129.csv'
# LHC sample of size 12000
#file_in = 'gem0py_new_20240123.csv'

modtype="gpr"
codename="gem0py"

### Read inputs
data = pd.read_csv(file_in)
data_x = data[xlabels]
data_y = data[ylabels]
x = np.array(data_x)

# Create scan files
print("Creating scan files")

remainder_file_path = f"scan_{codename}{modtype}_remainder_"
remainder_file_date = date_now
st_t = t.time()

scan_data = [[plot_gem0_scan(x,input_number=i,output_number=o, flag_plot=False, file_name_suf=date_now, extend_factor=0.05, \
                               cut_option='file', remainder_values=f"{remainder_file_path}{xlabels[i]}_{remainder_file_date}", \
                             ) for i in range(len(xlabels))] for o in range(len(ylabels))]
print("Time to create scan files: ", t.time()-st_t)

scan_data_perft = []

# Reshape dictionaries
for n_ft in range(n_fts):
     scan_data_perft.append({(f"{xlabels[i]}_{ylabels[j]}",k): scan_data[j][i][(f"ft{n_ft}",k)] for i,j,k in product(range(len(xlabels)),range(len(ylabels)),['x','y'])})	

# Write down scan files
#date_last = '20240116'

for n_ft in range(n_fts):
     s = scan_data_perft[n_ft]
     spd = pd.DataFrame.from_dict(s)
     spd.to_csv(f"scan_{codename}py_{date_now}_ft{n_ft}.csv")


Creating scan files
time to evaluate GEM0 for i0o0f0: 3.748338460922241
time to evaluate GEM0 for i0o0f1: 4.813377618789673
time to evaluate GEM0 for i0o0f2: 10.089278936386108
time to evaluate GEM0 for i0o0f3: 6.786756277084351
time to evaluate GEM0 for i0o0f4: 5.919029474258423
time to evaluate GEM0 for i0o0f5: 6.3129050731658936
time to evaluate GEM0 for i0o0f6: 6.789286851882935
time to evaluate GEM0 for i0o0f7: 7.505032062530518
time to evaluate GEM0 for i1o0f0: 3.6544735431671143
time to evaluate GEM0 for i1o0f1: 4.399113416671753
time to evaluate GEM0 for i1o0f2: 4.998999834060669
time to evaluate GEM0 for i1o0f3: 5.630670547485352
time to evaluate GEM0 for i1o0f4: 5.856913805007935
time to evaluate GEM0 for i1o0f5: 9.870150089263916
time to evaluate GEM0 for i1o0f6: 6.507187366485596
time to evaluate GEM0 for i1o0f7: 7.1080803871154785
time to evaluate GEM0 for i2o0f0: 3.884622573852539
time to evaluate GEM0 for i2o0f1: 4.376005411148071
time to evaluate GEM0 for i2o0f2: 4.9294

In [13]:
# # Test: reading a multi-indexed dataframe
# test_df = pd.read_csv("scan_gem0_remainder_te_value.csv", header=[0, 1], index_col=0,)

# test_df = test_df[(f"ft{0}", xlabels[0])]

# test_array = np.array(test_df)
# print(test_array)

[ 1807.38363671 -2654.58327936 -3295.23287111]


In [23]:
# To create surrogate scan files:

#!cd ../../../EasySurrogate && pip install . && cd tests/gem_gp && for((i=0;i<8;i++)); do python3 test_model_ann.py ${i} 20231218 && cp scan_${i}.csv ../../../MFW/uq/basicda/scan_gem0surr_20240105_ft${i}.csv ; done && cd ../../../MFW/uq/basicda

!cd ../../../EasySurrogate && pip install . && cd tests/gem_gp && for((i=0;i<8;i++)); do python3 test_model.py ${i} 20231212 && cp scan_${i}.csv ../../../MFW/uq/basicda/scan_gem0surr_20240105_ft${i}.csv ; done && cd ../../../MFW/uq/basicda


DEPRECATION: Loading egg at /cobra/u/yyudin/.local/lib/python3.11/site-packages/mogp_emulator-0.7.2-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..
DEPRECATION: Loading egg at /cobra/u/yyudin/conda-envs/python3114/lib/python3.11/site-packages/easysurrogate-0.18-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..
Processing /cobra/u/yyudin/code/EasySurrogate
  Preparing metadata (setup.py) ... done
  Created wheel for easysurrogate: filename=easysurrogate-0.24.1-py3-none-any.whl size=105687 sha256=2d5b8edc078b729106e29ffce107594d81ab9e81a04f19e7143ab1814b9ccadc
  Stored in directory: /tmp/pip-ephem-wheel-cache-y2hw69w9/wheels/a7/9b/db/505d51a3a6df8379c6b57380cb045f3d3a3237c864e8adaa7c
Successfully built easysurrogate
  Attempting uninstall: easysurrogate
    Found existing installation: easysurrogate 0.24.1
    Uninstal

In [12]:
# To plot the difference between GEM0 and surrogate, and their comparison

date = '20240129'
surtype = 'gpr'
codename = 'gem0py'

file_pref_gem0 = f"{codename}py_{date}"
file_pref_surr = f"{codename}{surtype}_{date}"

save_file = f"{codename}vs{surtype}_{date}_0.pdf"

plot_diff_all(file_pref_1=file_pref_gem0, file_pref_2=file_pref_surr, save_file=save_file)

!mv *gem0pyvsgpr_20240129_0.pdf ../../..

In [7]:
# To generate a dataset of LHCS GEM0 evaluations
#nft = 8
date = '20240125'

file_in = 'gem0py_new_20240123.csv'

save_filename=f"gem0py_lhc_{date}_ft"

n_samples_p_ft = 1500

for i in range(n_fts):
    
    print(f"flux tube number: {i}")

    write_gem0_offline(n_samples=n_samples_p_ft, n_ft=i, filename=save_filename, file_in=file_in)


flux tube number: 0
time to create extcodehelper: 0.4854755401611328
time to create LHC-S: 0.0014903545379638672
negative Ti:
[]
time to evaluate GEM0: 5.439159631729126
time to save GEM0: 0.026034116744995117
flux tube number: 1
time to create extcodehelper: 0.458362340927124
time to create LHC-S: 0.0014421939849853516
negative Ti:
[]
time to evaluate GEM0: 5.841173410415649
time to save GEM0: 0.02912735939025879
flux tube number: 2
time to create extcodehelper: 0.4724240303039551
time to create LHC-S: 0.0018677711486816406
negative Ti:
[]
time to evaluate GEM0: 6.904874563217163
time to save GEM0: 0.02989673614501953
flux tube number: 3
time to create extcodehelper: 0.5035548210144043
time to create LHC-S: 0.0017859935760498047
negative Ti:
[]
time to evaluate GEM0: 8.192617893218994
time to save GEM0: 0.04552960395812988
flux tube number: 4
time to create extcodehelper: 0.4966423511505127
time to create LHC-S: 0.0013761520385742188
negative Ti:
[]
time to evaluate GEM0: 8.4559710025

In [36]:
# To generate an expanded dataset of GEM0 evaluations

from da_utils import write_gem0_expanded

datenow = '20240126'
save_filename = f"gem0py_new_{datenow}.csv"

file_old = "gem_648_data.csv"

expand_factor = 1.0

write_gem0_expanded(file_old, save_filename, expand_factor=expand_factor, calloption=2)

n_points_perdim=3
n_p_p_ft_new=625
x_new = [[  648.17011543]
 [  401.47855654]
 [-4710.81464577]
 [-5847.70927745]]
x_new = [[  648.17011543]
 [  401.47855654]
 [-4710.81464577]
 [-4571.47107428]]
x_new = [[  648.17011543]
 [  401.47855654]
 [-4710.81464577]
 [-3295.23287111]]
x_new = [[  648.17011543]
 [  401.47855654]
 [-4710.81464577]
 [-2018.99466794]]
x_new = [[  648.17011543]
 [  401.47855654]
 [-4710.81464577]
 [ -742.75646477]]
x_new = [[  648.17011543]
 [  401.47855654]
 [-3682.69896256]
 [-5847.70927745]]
x_new = [[  648.17011543]
 [  401.47855654]
 [-3682.69896256]
 [-4571.47107428]]
x_new = [[  648.17011543]
 [  401.47855654]
 [-3682.69896256]
 [-3295.23287111]]
x_new = [[  648.17011543]
 [  401.47855654]
 [-3682.69896256]
 [-2018.99466794]]
x_new = [[  648.17011543]
 [  401.47855654]
 [-3682.69896256]
 [ -742.75646477]]
x_new = [[  648.17011543]
 [  401.47855654]
 [-2654.58327936]
 [-5847.70927745]]
x_new = [[  648.17011543]
 [  401.47855654]
 [-2654.58327936]
 [-4571.4710

0

In [8]:
# Add new readings to a training data set

from da_utils import merge_result_csv

code="gem0py"

date_old = "20240126"
date_new = "20240129"
date_now = "20240129"

file_old = f"{code}_new_{date_old}.csv"
file_new = f"test_{code}_{date_new}.csv"
file_tot = f"{code}_new_{date_now}.csv"

merge_result_csv([file_old, file_new], file_tot)


0

In [5]:
# To generate a dataset of GEM0 evaluations for given input file

from da_utils import write_gem0_fromfile

filename_in = 'new_surrogate_points_it98.csv'

filename_out = 'test_gem0py_20240129.csv'

write_gem0_fromfile(filename_in, filename_out)


/cobra/u/yyudin/code/MFW/uq/basicda/da_utils.py:413: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_out[input_names[j]].iloc[i] = x_new[j]
/cobra/u/yyudin/code/MFW/uq/basicda/da_utils.py:413: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_out[input_names[j]].iloc[i] = x_new[j]
/cobra/u/yyudin/code/MFW/uq/basicda/da_utils.py:413: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_out[input_names[j]].iloc[i] = x_new[j]
/cobra/u/yyudin/

0

In [8]:
# temporary: merge generated csv file into one, adding a column with the flux tube number

import glob

date = '20240125'
path = f"gem0py_lhc_{date}_ft*.csv" 

all_files = glob.glob(path)
df_from_each_file = [pd.read_csv(f) for f in all_files]

for i in range(len(df_from_each_file)):
    df_from_each_file[i]['ft'] = i

df_merged = pd.concat(df_from_each_file, ignore_index=True, sort=False)
df_merged.to_csv( f"gem0py_lhc_{date}_all.csv")


In [ ]:
# copy EasySurrogate campaign objects and replace 